# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902004


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:40,  1.44s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:49,  1.84s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:35,  1.37s/it]

Rendering models:  20%|██        | 6/30 [00:06<00:23,  1.02it/s]

...

Rendering models:  90%|█████████ | 27/30 [00:13<00:01,  2.76it/s]

Rendering models:  93%|█████████▎| 28/30 [00:13<00:00,  3.51it/s]

Rendering models:  97%|█████████▋| 29/30 [00:13<00:00,  3.55it/s]

Rendering models: 100%|██████████| 30/30 [00:13<00:00,  3.48it/s]

Jes8x                                 0.001146
CTidwell3                             0.000422
equidad1                              0.117043
not-logged-in-0965bc82e356a9e112d3    0.053198
daguta                                0.002192
cmk24                                 0.000356
kayleebug2017                         0.002376
mmcdonough4                           0.001022
not-logged-in-9a45911eb71184fe7c7f    0.003079
magedghoche                           0.002300
not-logged-in-256a8425122c4b10de2f    0.001637
AlexYoung35                           0.000358
clairedeu                             0.000712
scalise                               0.000920
pangeli5                              0.000624
not-logged-in-e1fe3d45a0f698d15856    0.000493
Lavadude                              0.002417
ShirleyL                              0.001185
not-logged-in-b17da99b1e7d1135ad5c    0.009624
ISlicedI                              0.000992
not-logged-in-e5015a872e4a0f43792d    0.413666
nicoletraher 

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())